In [40]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
from threading import Thread
from queuelib import queue
from selenium import webdriver
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [41]:
HOST = 'https://www.superbuy.com.tw/'

In [42]:
index_url = HOST

# 上層爬蟲

In [102]:
def SuperBuy_crawler():
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    rows = soup_index.select('.row')
    li_lists = rows[0].find('div')
    a_list = li_lists.select('a')
    
    for a in a_list:
        print(a.text)
        petten = 'category.php'
        pre_small_branch_url = a['href']
        regular_expression = re.match(petten,pre_small_branch_url)
        
        if regular_expression is not None:
            print(pre_small_branch_url)
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            print(small_branch_url)
            #pass
            #傳給中層
            small_branch_crawler(small_branch_url)    

# 中層爬蟲

In [103]:
def small_branch_crawler(small_branch_url):
    res = rq.get(small_branch_url)
    soup = bs(res.text, 'lxml')
    
    a_list = soup.select('.product.col-xs-6.col-sm-6.col-md-4 > ul > li > a["href"]')
    
    last_product_url = ''
    
    for a_href in a_list:
        pre_product_url = re.match(r'product\.php\?p\_id',a_href['href'])
        if pre_product_url is not None:
            product_url = "{}{}".format(index_url, a_href['href'])
            
            if product_url != last_product_url:
                print(product_url)
                #pass
                #傳給下層
                product_url_crawler(product_url)                
                last_product_url = product_url
                #print(last_product_url)

# 下層爬蟲

In [100]:
def product_url_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    
    #--右側欄位--
    #商品名稱
    title = soup.h1.text
    print(title)
    try:
        try:
            # 附標題
            sub_title = soup.select('span > strong > span')[0].text
            print(sub_title)
            # 商品說明
            productinfo = soup.select('.productinfo > p')[2].text
            print(productinfo)
            # 類別    
            category = soup.select('.productinfo > p > a')[0].text
            print(category)

        except IndexError:
            # 附標題
            sub_title = soup.select('span > span > strong')[0].text
            print(sub_title)
            # 商品說明
            productinfo = soup.select('.productinfo > div')[3].text
            print(productinfo)
            # 類別
            category = soup.select('.productinfo > p > a')[0].text
            print(category)

        except Exception:
            print('format_exception()')
    except Exception:
        tb.print_tb
            
    #--左側欄位--
    # 產品規格和注意事項
    try:
        try:
            div_Product_specifications = soup.select('div > ul > li > div')
            for div in div_Product_specifications:
                div_text = div.text.strip()

                # 成分
                ingredient = re.match(r'成份',div_text)
                if ingredient is not None:
                    print(div_text)

                # 原產地
                place_of_origin = re.match(r'原產地',div_text)
                if place_of_origin is not None:
                    print(div_text)

                # 保存期限
                retention_period = re.match(r'保存期限',div_text)
                if retention_period is not None:
                    print(div_text)

                # 保存方法
                preservation_method = re.match(r'保存方法',div_text)
                if preservation_method is not None:
                    print(div_text)

                #print(div_text)

        except IndexError:
            li_Product_specifications = soup.select('.column.full > ul')[0].select('li')
            for li in li_Product_specifications:
                li_text = li.text.strip()
                #print(li_text)
                #print(type(li_text))
                # 成分
                ingredient = re.match(r'成份',li_text)
                if ingredient is not None:
                    print(li_text)

                # 原產地
                place_of_origin = re.match(r'原產地',li_text)
                if place_of_origin is not None:
                    print(li_text)

                # 保存期限
                retention_period = re.match(r'保存期限',li_text)
                if retention_period is not None:
                    print(li_text)

                # 保存方法
                preservation_method = re.match(r'保存方法',li_text)
                if preservation_method is not None:
                    print(li_text)

                # 食用方法
                edible_method = re.match(r'食用方法',li_text)
                if edible_method is not None:
                    print(li_text)

                # 解凍方式
                thawing_method = re.match(r'解凍方式',li_text)
                if thawing_method is not None:
                    print(li_text)

                # 注意事項
                precautions = re.match(r'注意事項',li_text)
                if precautions is not None:
                    print(li_text)
    except Exception:
        tb.print_tb

# 下層爬蟲

In [184]:
# 黑鑽雞 去骨雞腿肉
product_url_test = 'https://www.superbuy.com.tw/product.php?p_id=1607040016'

In [195]:
# 有機轉型期金鑽鳳梨
product_url_test2 = 'https://www.superbuy.com.tw/product.php?p_id=1503250001'

In [196]:
# A級有機地瓜
product_url_test3 = 'https://www.superbuy.com.tw/product.php?p_id=1302190001'

In [197]:
res = rq.get(product_url_test3)
soup = bs(res.text, 'lxml')

# 右側欄位

## 商品名稱

In [116]:
soup.h1.text

'黑鑽雞 去骨雞腿肉'

In [ ]:
soup

In [90]:
# 附標題

In [ ]:
# A級有機地瓜

In [117]:
soup.select('span > strong > span')[0].text

IndexError: list index out of range

In [ ]:
# 有機轉型期金鑽鳳梨(4顆裝)

In [118]:
soup.select('span > span > strong')[0].text

'嚴選優質土雞品種 堅持自然完熟 低密度生長'

In [ ]:
# 商品說明

In [ ]:
# A級有機地瓜

In [119]:
soup.select('.productinfo > p')[2].text

'編號：0397-057'

In [ ]:
# 有機轉型期金鑽鳳梨(4顆裝)

In [120]:
soup.select('.productinfo > div')[3]

<div>
	來自雲林純淨牧場的頂級黑鑽雞，採無污染低密度高保濕的飼養環境培育，14週自然完熟，不施打成長激素、抗生素。每批肉品上市前都經過嚴格檢驗無藥殘，以及合格電宰廠屠宰、急速冷凍、冷凍宅配運送，讓消費者更放心享受雞肉豐潤鮮美的自然原味。肉質結實有彈性，少了一般肉雞的腥味，脂肪與含水量少，蛋白質更多、營養價值更高。皮薄彈牙、雞汁甘甜飽滿、肉質細緻口感佳。</div>

In [ ]:
# 類別

In [121]:
soup.select('.productinfo > p > a')[0].text

'雞肉'

# 以下在左邊欄位

In [ ]:
# 產品規格和注意事項

In [191]:
soup.select('.column.full > ul')[0].select('li')

[]

In [ ]:
# A級有機地瓜

In [204]:
div_soup.select('div > ul > li > div')

<div>
<img align="absMiddle" alt="" src="https://www.superbuy.com.tw/images/ico_orange.png"/> 成份：臺農57號地瓜</div>

In [206]:
div_Product_specifications = soup.select('div > ul > li > div')

In [214]:
for div in div_Product_specifications:
    div_text = div.text.strip()
    
    # 成分
    ingredient = re.match(r'成份',div_text)
    if ingredient is not None:
        print(div_text)
        
    # 原產地
    place_of_origin = re.match(r'原產地',div_text)
    if place_of_origin is not None:
        print(div_text)

    # 保存期限
    retention_period = re.match(r'保存期限',div_text)
    if retention_period is not None:
        print(div_text)
    
    # 保存方法
    preservation_method = re.match(r'保存方法',div_text)
    if preservation_method is not None:
        print(div_text)
    
    
    #print(div_text)

成份：臺農57號地瓜
原產地：台灣雲林
保存期限：2週~3週
保存方法：常溫，請放置陰涼通風處


In [ ]:
# 黑鑽雞 去骨雞腿肉

In [188]:
li_Product_specifications = soup.select('.column.full > ul')[0].select('li')

In [189]:
li_Product_specifications

[<li>成份：黑羽土雞</li>,
 <li>原產地：雲林</li>,
 <li>保存期限：冷凍-18℃以下，保存期限6個月</li>,
 <li>注意事項：家用冰箱冷凍庫因常開關取物，溫度難維持-18℃以下，因此建議保存期限約2～3個月</li>,
 <li>解凍方式：包裝密封狀態下，建議前一天宜置冷藏室採低溫解凍以保持食材原味，或以水沖泡快速解凍。解凍後請盡速食用完畢，請勿重複冷凍、解凍，以避免變質<br/></li>,
 <li>食用方法：依照個人喜好進行料理，可作湯品燉煮、烤雞</li>,
 <li>本公司所販售商品，屬於自然成長之肉品，其形狀、色澤、肥瘦皆不相同；肉品經過分切、去骨、修飾(剔除筋膜、油脂)後，標示重量皆會有微幅增減差異，屬肉品自然損耗，購買前敬請留意。<br/></li>]

In [ ]:
# 成分

In [180]:
for li in li_Product_specifications:
    li_text = li.text.strip()
    #print(li_text)
    #print(type(li_text))
    # 成分
    ingredient = re.match(r'成份',li_text)
    if ingredient is not None:
        print(li_text)
        
    # 原產地
    place_of_origin = re.match(r'原產地',li_text)
    if place_of_origin is not None:
        print(li_text)

    # 保存期限
    retention_period = re.match(r'保存期限',li_text)
    if retention_period is not None:
        print(li_text)
    
    # 保存方法
    preservation_method = re.match(r'保存方法',li_text)
    if preservation_method is not None:
        print(li_text)
    
    # 食用方法
    edible_method = re.match(r'食用方法',li_text)
    if edible_method is not None:
        print(li_text)
    
    # 解凍方式
    thawing_method = re.match(r'解凍方式',li_text)
    if thawing_method is not None:
        print(li_text)
    
    # 注意事項
    precautions = re.match(r'注意事項',li_text)
    if precautions is not None:
        print(li_text)
    

    
    

成份：黑羽土雞
原產地：雲林
保存期限：冷凍-18℃以下，保存期限6個月
注意事項：家用冰箱冷凍庫因常開關取物，溫度難維持-18℃以下，因此建議保存期限約2～3個月
解凍方式：包裝密封狀態下，建議前一天宜置冷藏室採低溫解凍以保持食材原味，或以水沖泡快速解凍。解凍後請盡速食用完畢，請勿重複冷凍、解凍，以避免變質
食用方法：依照個人喜好進行料理，可作湯品燉煮、烤雞
